In [ ]:
import pathlib
import tempfile
from urllib.parse import urlparse
from urllib.request import urlretrieve

import numpy as np
import asdf
from astropy.io import fits
from astropy.modeling import models, fitting
import astropy.units as u
import matplotlib.pyplot as plt

from gwcs import wcs as gwcs
from gwcs import coordinate_frames as cf

from grism_observation import GrismObs
from HST.hst_grism_reffiles import create_tsgrism_wavelengthrange, create_grism_specwcs
from HST.generate_wfc3_distortion import create_wfc3_distortion

In [ ]:
reference_files = dict()

conf_filepath = "HST/UVIS_G280_CCD1_V2.conf"

filter = "G280"

specwcs_filename = "WFC3_" + str(filter) + "_specwcs.asdf"
create_grism_specwcs(conffile=str(conf_filepath), pupil=filter, outname=specwcs_filename)

#### Create inverse SIP coefficients for UVIS

In [ ]:
sip_file = "/Users/rosteen/Data/UVIS/G280/icwz15e7q_flt.fits"
sip_hdus = fits.open(str(sip_file))

# CCD 1
acoef_1 = dict(sip_hdus[1].header['A_*'])
a_order_1 = acoef_1.pop('A_ORDER')
bcoef_1 = dict(sip_hdus[1].header['B_*'])
b_order_1 = bcoef_1.pop('B_ORDER')

# CCD 2
acoef_2 = dict(sip_hdus[4].header['A_*'])
a_order_2 = acoef_2.pop('A_ORDER')
bcoef_2 = dict(sip_hdus[4].header['B_*'])
b_order_2 = bcoef_2.pop('B_ORDER')

In [ ]:
acoef_1

In [ ]:
acoef_2

In [ ]:
sip_hdus[1].data.shape

In [ ]:
indices = np.indices((2051, 4096))
print(indices)

In [ ]:
indices.shape

In [ ]:
x_indices = indices[0,:]
y_indices = indices[1,:]

In [ ]:
a_polycoef_1 = {}
for key in acoef_1:
    a_polycoef_1['c' + key.split('A_')[1]] = acoef_1[key]

b_polycoef_1 = {}
for key in bcoef_1:
    b_polycoef_1['c' + key.split('B_')[1]] = bcoef_1[key]
    
a_polycoef_2 = {}
for key in acoef_2:
    a_polycoef_2['c' + key.split('A_')[1]] = acoef_2[key]

b_polycoef_2 = {}
for key in bcoef_2:
    b_polycoef_2['c' + key.split('B_')[1]] = bcoef_2[key]

In [ ]:
a_poly_1 = models.Polynomial2D(a_order_1, **a_polycoef_1)
b_poly_1 = models.Polynomial2D(b_order_1, **b_polycoef_1)

a_poly_2 = models.Polynomial2D(a_order_2, **a_polycoef_2)
b_poly_2 = models.Polynomial2D(b_order_2, **b_polycoef_2)

In [ ]:
result_x_1 = a_poly_1(x_indices, y_indices)
result_y_1 = b_poly_1(x_indices, y_indices)

result_x_2 = a_poly_2(x_indices, y_indices)
result_y_2 = b_poly_2(x_indices, y_indices)

In [ ]:
result_x_1

In [ ]:
x_indices + result_x_1

In [ ]:
# Initialize a 5th degree 2D polynomial for fitting
inverse_model = models.Polynomial2D(5)

In [ ]:
inverse_model

In [ ]:
fitter = fitting.LevMarLSQFitter()

best_fit = fitter(inverse_model, x_indices+result_x_1, y_indices+result_y_1, -result_x_1)
x_inverse_model = best_fit

best_fit = fitter(inverse_model, x_indices+result_x_1, y_indices+result_y_1, -result_y_1)
y_inverse_model = best_fit

In [ ]:
x_inverse_model

In [ ]:
y_inverse_model

In [ ]:
x_inverse_model(x_indices+result_x_1, y_indices+result_y_1)

In [ ]:
(x_inverse_model(x_indices+result_x_1, y_indices+result_y_1) + result_x_1).max()

In [ ]:
(y_inverse_model(x_indices+result_x_1, y_indices+result_y_1) + result_y_1).max()

In [ ]:
fitter = fitting.LevMarLSQFitter()

best_fit = fitter(inverse_model, x_indices+result_x_2, y_indices+result_y_2, -result_x_2)
x_inverse_model_2 = best_fit

best_fit = fitter(inverse_model, x_indices+result_x_2, y_indices+result_y_2, -result_y_2)
y_inverse_model_2 = best_fit

In [ ]:
# Testing to see if this actually worked...

crpix = [sip_hdus[1].header['CRPIX1'], sip_hdus[1].header['CRPIX2']]

crval = [sip_hdus[1].header['CRVAL1'],
         sip_hdus[1].header['CRVAL2']]

cdmat = np.array([[sip_hdus[1].header['CD1_1'], sip_hdus[1].header['CD1_2']],
                  [sip_hdus[1].header['CD2_1'], sip_hdus[1].header['CD2_2']]])

SIP_forward = (models.Shift(-(crpix[0]-1)) & models.Shift(-(crpix[1]-1)) | # Calculate u and v
              models.Mapping((0, 1, 0, 1, 0, 1)) | a_poly_1 & b_poly_1 & models.Identity(2) |
              models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
              models.AffineTransformation2D(matrix=cdmat) | models.Pix2Sky_TAN() |
              models.RotateNative2Celestial(crval[0], crval[1], 180))

SIP_backward = (models.RotateCelestial2Native(crval[0], crval[1], 180) |
             models.Sky2Pix_TAN() | models.AffineTransformation2D(matrix=cdmat).inverse |
             models.Mapping((0, 1, 0, 1, 0, 1)) | x_inverse_model & y_inverse_model & models.Identity(2) |
             models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
             models.Shift((crpix[0]-1)) & models.Shift((crpix[1]-1)))

full_distortion_model = SIP_forward
full_distortion_model.inverse = SIP_backward

imagepipe = []

det_frame = cf.Frame2D(name="detector")
imagepipe.append((det_frame, full_distortion_model))
world_frame = cf.CelestialFrame(name="world", unit = (u.Unit("deg"), u.Unit("deg")),
                                axes_names=('lon', 'lat'), axes_order=(0, 1),
                                reference_frame="ICRS")
imagepipe.append((world_frame, None))
geo_transforms = gwcs.WCS(imagepipe)

In [ ]:
geo_transforms.available_frames

In [ ]:
forward = geo_transforms.get_transform("detector", "world")

In [ ]:
backward = geo_transforms.get_transform("world", "detector")

In [ ]:
forward(2048, 1026)

In [ ]:
backward(206.43128029664106, 26.4186090137141)

In [ ]:
# Now test the models for CCD2

crpix = [sip_hdus[4].header['CRPIX1'], sip_hdus[4].header['CRPIX2']]

crval = [sip_hdus[4].header['CRVAL1'],
         sip_hdus[4].header['CRVAL2']]

cdmat = np.array([[sip_hdus[4].header['CD1_1'], sip_hdus[4].header['CD1_2']],
                  [sip_hdus[4].header['CD2_1'], sip_hdus[4].header['CD2_2']]])

SIP_forward = (models.Shift(-(crpix[0]-1)) & models.Shift(-(crpix[1]-1)) | # Calculate u and v
              models.Mapping((0, 1, 0, 1, 0, 1)) | a_poly_2 & b_poly_2 & models.Identity(2) |
              models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
              models.AffineTransformation2D(matrix=cdmat) | models.Pix2Sky_TAN() |
              models.RotateNative2Celestial(crval[0], crval[1], 180))

SIP_backward = (models.RotateCelestial2Native(crval[0], crval[1], 180) |
             models.Sky2Pix_TAN() | models.AffineTransformation2D(matrix=cdmat).inverse |
             models.Mapping((0, 1, 0, 1, 0, 1)) | x_inverse_model_2 & y_inverse_model_2 & models.Identity(2) |
             models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
             models.Shift((crpix[0]-1)) & models.Shift((crpix[1]-1)))

full_distortion_model = SIP_forward
full_distortion_model.inverse = SIP_backward

imagepipe = []

det_frame = cf.Frame2D(name="detector")
imagepipe.append((det_frame, full_distortion_model))
world_frame = cf.CelestialFrame(name="world", unit = (u.Unit("deg"), u.Unit("deg")),
                                axes_names=('lon', 'lat'), axes_order=(0, 1),
                                reference_frame="ICRS")
imagepipe.append((world_frame, None))
geo_transforms = gwcs.WCS(imagepipe)

In [ ]:
forward = geo_transforms.get_transform("detector", "world")
backward = geo_transforms.get_transform("world", "detector")

In [ ]:
forward(2048, 1026)

In [ ]:
backward(206.4568554451976, 26.418285311669543)

In [ ]:
x_inverse_model_2

In [ ]:
y_inverse_model_2

In [ ]:
y_inverse_model_2.parameters

In [ ]:
y_inverse_model_2.param_names

In [ ]:
getattr(y_inverse_model_2, "c0_0").value

In [ ]:
# Loop through the models to get the inverse coefficients AP_* and BP_*
from datetime import datetime
ir_sip = fits.open("config/HST/WFC3_IR_distortion.fits")

hdus = fits.HDUList()
hdus.append(ir_sip[0])
hdus.append(fits.ImageHDU(header=fits.Header(), data=None))
hdus.append(fits.ImageHDU(header=fits.Header(), data=None))

hdr = hdus[0].header
hdr["MADEBY"] = "Ricky O'Steen"
hdr["DETECTOR"] = "UVIS"
hdr["MADEON"] = str(datetime.now())
hdr["NCHIP"] = 2
hdus[0].header = hdr

In [ ]:
hdus[0].header

In [ ]:
# CCD 2 (comes first, in HDU 1 for some reason)

hdr = hdus[1].header
hdr["EXTNAME"] = "UVIS"
hdr["CHIPNAME"] = "UVIS"
hdr["CCDCHIP"] = sip_hdus[1].header["CCDCHIP"]

hdr["A_ORDER"] = a_order_1
for key in acoef_1:
    hdr[key] = acoef_1[key]
    
hdr["B_ORDER"] = b_order_1
for key in bcoef_1:
    hdr[key] = bcoef_1[key]
    
hdr["AP_ORDER"] = 5
for param in x_inverse_model.param_names:
    key = "AP_{}_{}".format(param[1], param[3])
    hdr[key] = getattr(x_inverse_model, param).value
    
hdr["BP_ORDER"] = 5
for param in y_inverse_model.param_names:
    key = "BP_{}_{}".format(param[1], param[3])
    hdr[key] = getattr(y_inverse_model, param).value

hdus[1].header = hdr

# CCD 1 (they're in reverse order in the original file)

hdr = hdus[2].header
hdr["EXTNAME"] = "UVIS"
hdr["CHIPNAME"] = "UVIS"
hdr["CCDCHIP"] = sip_hdus[4].header["CCDCHIP"]

hdr["A_ORDER"] = a_order_2
for key in acoef_2:
    hdr[key] = acoef_2[key]
    
hdr["B_ORDER"] = b_order_2
for key in bcoef_2:
    hdr[key] = bcoef_2[key]
    
hdr["AP_ORDER"] = 5
for param in x_inverse_model_2.param_names:
    key = "AP_{}_{}".format(param[1], param[3])
    hdr[key] = getattr(x_inverse_model_2, param).value
    
hdr["BP_ORDER"] = 5
for param in y_inverse_model_2.param_names:
    key = "BP_{}_{}".format(param[1], param[3])
    hdr[key] = getattr(y_inverse_model_2, param).value

hdus[2].header = hdr

In [ ]:
hdus[1].header

In [ ]:
hdus[2].header

In [ ]:
hdus.writeto("config/HST/WFC3_UVIS_distortion.fits", overwrite=True)

#### Testing backward and forward dispersion to confirm round trip

In [ ]:
# Try out a GrismObs for UVIS with no INVDISPY defined to see if we can get anything

sip_file = "/Users/rosteen/Data/UVIS/G280/icwz15e7q_flt.fits"

test = GrismObs(sip_file)

In [ ]:
import asdf
from HST.dispersion_models import DISPXY_Extension

asdf.get_config().add_extension(DISPXY_Extension())

In [ ]:
specwcs = asdf.open("config/HST/WFC3_G280_specwcs.asdf").tree
print(specwcs.keys())
print(specwcs["order"])

In [ ]:
temp_model = specwcs['displ'][0]
temp_model

In [ ]:
temp_model.n_inputs

In [ ]:
test.geometric_transforms.available_frames

In [ ]:
g2d = test.geometric_transforms.get_transform("grism_detector", "detector")

In [ ]:
d2g = test.geometric_transforms.get_transform("detector", "grism_detector")

In [ ]:
g2d

In [ ]:
g2d(-111.09132139180039, 281.5373678242843, 105.0, 105.0, 1.0)

In [ ]:
d2g(105.0, 105.0, 4500, 1.0)

In [ ]:
g2d.has_inverse()

In [ ]:
d2g.has_inverse()

In [ ]:
temp_model(100,100,.6)

In [ ]:
temp_model = specwcs['dispx'][1]

In [ ]:
temp_model(100,100,10)

#### Sanity checking inverse X dispersion for UVIS

In [ ]:
e_x = np.array((10.0, -570.0))
e_x

In [ ]:
if e_x.shape == (2,):
    e_x = np.reshape(e_x, [2,1])
e_x

In [ ]:
e_x.shape

In [ ]:
if len(e_x.shape) > 1:
    print(e_x.shape[1] > 2)

In [ ]:
offset = 0
e = e_x
x = 100
y = 50
t = -218
coeffs = {1: np.array([1]),
          6: np.array([1, x, y, x**2, x*y, y**2])}
t_order = e.shape[0]
if len(e.shape) == 1:
    c_order = 1
else:
    c_order = e.shape[1]
print(coeffs)
print(t_order)
print(c_order)

In [ ]:
((t + offset - np.dot(coeffs[c_order], e[0,:])) / np.dot(coeffs[c_order], e[1,:]))

In [ ]:
f = 0
t = 0.4
for i in range(0, t_order):
    f += t**i * (np.dot(coeffs[c_order], e[i,:]))
print(f)

Well, that seems to round trip correctly between dx and t. So the problem isn't there. 

In [ ]:
lmodel = temp_model

In [ ]:
lmodel(100, 50, 0.4)

In [ ]:
lmodel.ematrix.shape

#### It seems like split_order_info is dropping higher orders in t, investigating:

In [ ]:
keydict = {'NAXIS': (4096, 2048), 'DISPL_+1_0': (1247.907, 0.1715171, 0.4014437, 4.60608e-06, -0.0002067114, 1.051219e-05), 'DISPL_+1_1': (16183.52, -5.25768, -12.41063, -8.086326e-05, 0.0063084, -0.0002608619), 'DISPL_+1_2': (-94033.28, 52.1532, 113.8204, 0.0002302944, -0.05746772, 0.001580457), 'DISPL_+1_3': (369916.3, -195.763, -417.8535, 0.000160051, 0.2091686, -0.003993138), 'DISPL_+1_4': (-592296.5, 307.5212, 648.8591, -0.001038922, -0.3231617, 0.004620623), 'DISPL_+1_5': (327729.5, -168.2244, -353.9009, 0.0006380609, 0.1757735, -0.002072159), 'DISPX_+1_0': 10.0, 'DISPX_+1_1': -570.0, 'DISPY_+1_0': (199.1281, -0.002073556, 0.007069947, 7.430371e-08, -5.304913e-07, -1.286096e-06), 'DISPY_+1_1': (-176.623, -0.04297293, -0.1854369, 2.078917e-06, 4.561426e-05, 4.474414e-05), 'DISPY_+1_2': (96.46943, 0.3552698, 1.989722, -1.063473e-05, -0.0005489048, -0.0004790314), 'DISPY_+1_3': (2448.071, -1.083281, -9.225543, 6.593498e-05, 0.003044477, 0.002626904), 'DISPY_+1_4': (-9312.729, 0.9810426, 19.87687, -0.0005298376, -0.009131209, -0.008037482), 'DISPY_+1_5': (17329.54, 2.076095, -12.74858, 0.002099644, 0.01501813, 0.01349792), 'DISPY_+1_6': (-22863.98, -6.765774, -20.1584, -0.003786586, -0.01120987, -0.01117688), 'DISPY_+1_7': (22666.73, 6.63481, 26.92799, 0.002336442, -0.0006700353, 0.003283768), 'DISPY_+1_8': (-3105.089, 2.521005, 14.0929, 0.001538533, 0.00306462, -0.0005739407), 'DISPY_+1_9': (-24780.85, -10.98437, -22.67926, -0.002123915, 0.00537769, 0.001402719), 'DISPY_+1_10': (11342.89, 3.72694, -14.05894, -0.0005578266, -0.005013537, -0.002728773), 'DISPY_+1_11': (29798.37, 11.56107, 16.60022, 0.001130154, -0.004212057, 0.007544599), 'DISPY_+1_12': (-33431.68, -13.33704, 4.525097, 9.792405e-05, 0.006175275, -0.008287286), 'DISPY_+1_13': (9974.387, 4.360017, -4.950169, -0.000262393, -0.001940865, 0.002881733), 'DISPL_+2_0': (2024.832, -0.2083087, -0.3514176, 1.358243e-05, 0.0001676521, 3.734842e-06), 'DISPL_+2_1': (-1111.281, 2.246752, 3.163863, -0.0001426102, -0.001627307, 1.358261e-05), 'DISPL_+2_2': (11591.32, -6.665012, -9.389449, 0.0004553896, 0.004905473, -0.0001097879), 'DISPL_+2_3': (-11690.4, 6.977159, 9.703328, -0.0004778925, -0.005134452, 0.0001640513), 'DISPX_+2_0': -110.0, 'DISPX_+2_1': -440.0, 'DISPY_+2_0': (204.141, -0.001035233, 0.005817347, -4.270251e-07, -3.155895e-07, -9.469504e-07), 'DISPY_+2_1': (-150.8561, -0.07765548, 0.05642477, 2.473683e-05, -3.195813e-06, -2.229836e-06), 'DISPY_+2_2': (1191.992, 1.319842, -1.778713, -0.0004181253, 0.0001586993, 0.0003081203), 'DISPY_+2_3': (-6940.575, -11.84139, 15.85926, 0.003396008, -0.0007767892, -0.003469559), 'DISPY_+2_4': (19694.26, 59.88403, -67.57421, -0.01528851, -0.0007811398, 0.01741981), 'DISPY_+2_5': (-23600.21, -174.6648, 164.5273, 0.04041565, 0.01360129, -0.04932173), 'DISPY_+2_6': (-894.8858, 299.9207, -243.0514, -0.06418066, -0.0381387, 0.08370371), 'DISPY_+2_7': (29852.27, -299.1559, 216.755, 0.06019354, 0.04916766, -0.08417285), 'DISPY_+2_8': (-26690.85, 160.2556, -107.8316, -0.03071801, -0.03083378, 0.04612765), 'DISPY_+2_9': (7517.753, -35.63617, 23.04743, 0.006575413, 0.00760293, -0.01059303), 'DISPL_+3_0': (1812.834, -0.08144435, -0.1637695, 9.726739e-06, 6.464318e-05, 1.539862e-05), 'DISPL_+3_1': (501.0111, 1.173575, 1.677, -8.70214e-05, -0.0007949958, -6.536542e-05), 'DISPL_+3_2': (4678.522, -2.560219, -3.966128, 0.000163126, 0.001923605, 4.478018e-05), 'DISPX_+3_0': -230.0, 'DISPX_+3_1': -570.0, 'DISPY_+3_0': (217.2404, -0.004404165, -0.002326764, -5.134273e-08, 1.425584e-06, 1.380885e-06), 'DISPY_+3_1': (-458.1283, 0.09622443, 0.3678026, 3.681068e-06, -8.673845e-05, -6.997455e-05), 'DISPY_+3_2': (3421.222, -0.4734845, -2.917941, -0.000195094, 0.0009067482, 0.0002752907), 'DISPY_+3_3': (-10772.58, -1.583463, 7.719787, 0.001872398, -0.003793024, 0.001281324), 'DISPY_+3_4': (10845.43, 16.69144, -0.2409367, -0.007452941, 0.007001864, -0.01040463), 'DISPY_+3_5': (10811.63, -43.5201, -29.09555, 0.0143422, -0.004830692, 0.02457058), 'DISPY_+3_6': (-27786.6, 47.13105, 42.75724, -0.01324685, -0.0006884939, -0.02477995), 'DISPY_+3_7': (14184.06, -18.56394, -18.80972, 0.004716432, 0.001549921, 0.00919549), 'DISPL_+4_0': (1600.949, 0.1213913, 0.3136126, -4.082954e-06, -8.972904e-05, -0.0001056551), 'DISPL_+4_1': (1817.333, -0.4031969, -0.8642205, 1.656113e-05, 0.0003613914, 8.784479e-05), 'DISPX_+4_0': -400.0, 'DISPX_+4_1': -300.0, 'DISPY_+4_0': (210.4929, 4.920439e-05, 0.005616888, -6.255189e-07, -6.104241e-07, -4.258491e-07), 'DISPY_+4_1': (-195.1789, 0.05993905, 0.1968366, -5.779052e-06, -3.044978e-05, -4.909591e-05), 'DISPY_+4_2': (774.8928, -0.2793951, -0.852847, 3.351429e-05, 0.0001265894, 0.0002197228), 'DISPY_+4_3': (-1033.047, 0.3729296, 1.225291, -5.166284e-05, -0.0001587204, -0.0003370864), 'DISPY_+4_4': (447.3251, -0.1545797, -0.5696233, 2.530197e-05, 5.909885e-05, 0.0001701652), 'DISPL_-1_0': (1549.674, 0.112042, 0.06417442, -6.147326e-06, -5.088009e-05, -6.847407e-06), 'DISPL_-1_1': (11869.88, -1.247266, -1.489307, 0.0001672497, 0.0006572729, 0.0001935824), 'DISPL_-1_2': (-25774.4, 12.05035, 13.68945, -0.001581738, -0.006179599, -0.001627053), 'DISPL_-1_3': (83199.87, -44.74023, -52.13631, 0.006030946, 0.0230891, 0.005859683), 'DISPL_-1_4': (-121352.4, 70.31003, 81.9733, -0.009726034, -0.03591739, -0.008986659), 'DISPL_-1_5': (64076.54, -38.90405, -44.93107, 0.005525686, 0.01956408, 0.004864258), 'DISPX_-1_0': 230.0, 'DISPX_-1_1': 550.0, 'DISPY_-1_0': (188.3974, -0.003754053, 0.004924201, 4.759416e-08, 4.362399e-08, -5.989863e-07), 'DISPY_-1_1': (-613.0667, -0.0196904, -0.09794328, 6.600423e-06, -5.357835e-06, 4.602359e-05), 'DISPY_-1_2': (5133.404, 0.83428, 0.9635349, -8.6046e-05, 0.0001535844, -0.0006630445), 'DISPY_-1_3': (-27995.75, -8.547256, -4.47171, 0.0002591853, -0.001450286, 0.00407869), 'DISPY_-1_4': (94426.94, 42.75522, 10.93902, 0.001520206, 0.007474118, -0.0124392), 'DISPY_-1_5': (-191737.5, -118.9323, -18.18947, -0.01329702, -0.02470405, 0.01661289), 'DISPY_-1_6': (204827.6, 186.3674, 34.63798, 0.03979646, 0.05482801, 0.008509309), 'DISPY_-1_7': (-25571.73, -146.2479, -62.73186, -0.05463689, -0.07985152, -0.07105488), 'DISPY_-1_8': (-209032.8, 17.41713, 44.20711, 0.01989377, 0.07204076, 0.1215042), 'DISPY_-1_9': (229453.2, 60.65767, 61.53437, 0.03378501, -0.04029665, -0.08961658), 'DISPY_-1_10': (-49532.57, -54.94505, -147.3389, -0.03499904, 0.0249539, -0.009630011), 'DISPY_-1_11': (-73536.2, 37.81011, 114.5701, -0.002604193, -0.02728788, 0.07108235), 'DISPY_-1_12': (56728.44, -24.37575, -38.11669, 0.01581248, 0.01925721, -0.05054049), 'DISPY_-1_13': (-12615.01, 7.228475, 4.082948, -0.005450909, -0.005111253, 0.01211345), 'DISPL_-2_0': (1663.849, 0.04262151, -0.008425576, 1.089535e-06, -1.426305e-05, -3.244403e-06), 'DISPL_-2_1': (4192.888, 0.1979919, 0.3280327, 6.240839e-06, -0.0001375091, 3.00115e-05), 'DISPL_-2_2': (1737.929, -0.7388151, -2.889202, -0.0003364382, 0.00124591, -2.124256e-06), 'DISPL_-2_3': (-2465.904, -1.462081, 4.50913, 0.00152215, -0.002077705, -2.861282e-05), 'DISPX_-2_0': 350.0, 'DISPX_-2_1': 550.0, 'DISPY_-2_0': (179.7632, -0.003749091, 0.00308422, 7.811054e-08, -1.498714e-09, 7.437671e-08), 'DISPY_-2_1': (-284.9476, -0.0007426225, -0.04874903, 8.816693e-06, -2.309535e-05, 3.468432e-05), 'DISPY_-2_2': (1119.395, 0.1476601, 0.3157227, -0.0001268492, 0.0004291934, -0.0004920665), 'DISPY_-2_3': (-3115.988, -1.310789, -0.5050898, 0.0007984248, -0.003106123, 0.002870848), 'DISPY_-2_4': (5305.771, 5.521898, -2.561995, -0.002763279, 0.01233817, -0.009297641), 'DISPY_-2_5': (-5883.628, -12.41353, 12.67715, 0.00562105, -0.02991129, 0.01911526), 'DISPY_-2_6': (6250.749, 14.60777, -23.71703, -0.006667333, 0.04527629, -0.02601262), 'DISPY_-2_7': (-7902.364, -7.555967, 23.28939, 0.004278266, -0.04162732, 0.02253373), 'DISPY_-2_8': (6714.891, 0.1409697, -12.21075, -0.001193608, 0.02123268, -0.011062), 'DISPY_-2_9': (-2260.354, 0.8597784, 2.751915, 4.486396e-05, -0.004605215, 0.002309556), 'DISPL_-3_0': (1699.096, 0.04791965, 0.005768717, 1.349425e-06, -1.79641e-05, 2.70063e-06), 'DISPL_-3_1': (2241.699, -0.005076667, -0.06030019, -9.272839e-06, 3.117269e-05, -9.086008e-06), 'DISPL_-3_2': (-127.2566, 0.01978762, 0.02508017, 8.402054e-06, -2.274952e-05, 7.800285e-06), 'DISPX_-3_0': 475.0, 'DISPX_-3_1': 425.0, 'DISPY_-3_0': (164.4988, -0.0004179673, 0.003403746, -1.212223e-07, -1.044905e-06, 5.146071e-07), 'DISPY_-3_1': (-42.24046, -0.05480477, -0.01330544, 6.772708e-06, 7.612704e-06, -2.465134e-06), 'DISPY_-3_2': (-320.522, 0.368742, -0.1468771, -4.071e-05, 2.743191e-05, 1.537238e-05), 'DISPY_-3_3': (1358.063, -1.179057, 1.361773, 9.840595e-05, -0.000319236, -0.0001596528), 'DISPY_-3_4': (-2606.893, 2.237403, -4.580686, -0.0001216446, 0.0009657522, 0.0007606344), 'DISPY_-3_5': (2803.869, -2.708439, 7.477189, 0.0001053406, -0.001336005, -0.001593903), 'DISPY_-3_6': (-1690.183, 1.93912, -5.910014, -8.51805e-05, 0.0008800377, 0.001490757), 'DISPY_-3_7': (461.4511, -0.6149833, 1.80622, 3.882204e-05, -0.0002208068, -0.0005133015), 'DISPL_-4_0': (1842.831, 0.07204487, 0.02150832, -2.672041e-06, -2.824821e-05, 4.433913e-06), 'DISPL_-4_1': (1468.023, -0.08327019, -0.1233711, 1.624981e-06, 6.942547e-05, -1.145358e-05), 'DISPX_-4_0': 640.0, 'DISPX_-4_1': 350.0, 'DISPY_-4_0': (151.4997, -0.003560823, 0.003106131, 3.759473e-07, -4.151129e-08, -7.577361e-07), 'DISPY_-4_1': (-34.41476, 0.00876537, -0.04449884, -5.821345e-06, 6.682905e-06, 1.338941e-05), 'DISPY_-4_2': (-93.80154, 0.01014785, 0.1131061, 1.985179e-05, -5.029344e-05, -1.161397e-05), 'DISPY_-4_3': (226.5722, -0.06508698, -0.08721553, -2.932264e-05, 0.0001178812, -5.373507e-05), 'DISPY_-4_4': (-144.8692, 0.04896433, 0.01273369, 1.585546e-05, -7.752604e-05, 5.724802e-05)}

In [ ]:
import re
import numpy as np

token = re.compile('^[a-zA-Z]*_(?:[+\-]){0,1}[a-zA-Z0-9]{0,1}_*')
rangekey = re.compile('^[a-zA-Z]*_[0-1]{1,1}$')
rdict = dict()  # return dictionary
beams = list()

for key in keydict:
    if key[0:5] == "WEDGE":
        if "WEDGE" not in beams:
            beams.append("WEDGE")
    elif token.match(key):
        b = key.split("_")[1].upper()
        if b not in beams:
            beams.append(b)
for b in beams:
    rdict[b] = dict()
print(beams)

In [ ]:
for key in keydict:
    # Again, reject WEDGE keys
    if key[0:5] == "WEDGE":
        b, fname = key.split("_")
        rdict[b][fname] = keydict[key]
    elif token.match(key):
        b = key.split("_")[1].upper()
        newkey = key.replace("_{}_".format(b), "_")
        rdict[b][newkey] = keydict[key]
        print(f"{key} matches token")
    else:
        print(f"{key} doesn't match token")

In [ ]:
print(rdict)

In [ ]:
rangekey = re.compile('^[a-zA-Z]*_[0-1]{0,1}[0-9]{1,1}$')

for b, d in rdict.items():
    keys = d.keys()
    rkeys = []
    odict = {}
    for k in keys:
        if rangekey.match(k):
            rkeys.append(k)
        else:
            print(f"{k} doesn't match rangekey")
            
    for k in rkeys:
        mlist = [m for m in rkeys if k.split("_")[0] in m]
        print(mlist)
        root = mlist[0].split("_")[0]
        print(root)
        if root not in odict:
            for mk in mlist:
                if eval(mk[-1]) == 0:
                    zero = d[mk]
                elif eval(mk[-1]) == 1:
                    one = d[mk]
                else:
                    print("Would error")
            odict[root] = (zero, one)
            print(odict[root])

That was in fact the case. It's fixed now and properly setting e.g. DISPL to a 4x6 array where appropriate

In [ ]:
lmodel.evaluate(100, 50, 0.4)

In [ ]:
t = np.linspace(0, 1, 40)

In [ ]:
l = lmodel.evaluate(100, 500, t)
l

In [ ]:
import matplotlib.pyplot as plt
plt.plot(l)

#### To compare with GRISMCONF

In [ ]:
from grismconf import Config

In [ ]:
C = Config("/Users/rosteen/projects/GRISM_WFC3/UVIS/UVIS_G280_CCD1_V2.conf")

In [ ]:
w0,w1 = C.WRANGE["+1"] # wav range of order
print(w0, w1)

In [ ]:
t0,t1 = C.INVDISPL("+1",100,100,[w0,w1]) #  validd ts  values for order
print(t0, t1)

In [ ]:
xs = [C.DISPX("+1",100,100,x) for x in np.arange(t0,t1,0.01)]
lam = [C.DISPL("+1",100,100,x) for x in np.arange(t0,t1,0.01)]
plt.plot(xs,lam)

In [ ]:
l = lmodel.evaluate(100, 100, np.arange(t0,t1,0.01))
plt.plot(xs, l)

In [ ]:
diff = np.array(lam) - l
plt.plot(xs, diff)

In [ ]:
lmodel.ematrix

In [ ]:
C.orders

In [ ]:
xs2 = specwcs['dispx'][0](100,100,np.arange(t0,t1,0.01))

In [ ]:
plt.plot(xs2)

In [ ]:
plt.plot(xs)

In [ ]:
plt.plot(np.array(xs)-np.array(xs2))

In [ ]:
ys2 = specwcs['dispy'][0](100,100,np.arange(t0,t1,0.01))

In [ ]:
ys = [C.DISPY("+1",100,100,x) for x in np.arange(t0,t1,0.01)]

In [ ]:
ys - ys2